In [1]:
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision as tv
from torch.utils.data import DataLoader,Dataset

In [2]:
df = pd.read_csv("heart_failure_clinical_records_dataset.csv")
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
print(X)

      age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0    75.0        0                       582         0                 20   
1    55.0        0                      7861         0                 38   
2    65.0        0                       146         0                 20   
3    50.0        1                       111         0                 20   
4    65.0        1                       160         1                 20   
..    ...      ...                       ...       ...                ...   
294  62.0        0                        61         1                 38   
295  55.0        0                      1820         0                 38   
296  45.0        0                      2060         1                 60   
297  45.0        0                      2413         0                 38   
298  50.0        0                       196         0                 45   

     high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \

In [5]:
X = X.values

In [6]:
y = (y.values).reshape(-1,1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [8]:
X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [9]:
mean = X_train.mean()
std = X_train.std()
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [10]:
n_features = X_train.shape[1]
print(n_features)

12


In [11]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super().__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
model = LogisticRegression(n_features)

In [12]:
criterion = nn.BCELoss()
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(),lr)

In [13]:
model = LogisticRegression(n_features)

In [14]:
num_epochs = 100
for epoch in range(num_epochs):
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch + 1) % 10 == 0:
        print(f"epoch:{epoch + 1}, loss:{loss.item():.4f}")

epoch:10, loss:0.6461
epoch:20, loss:0.6461
epoch:30, loss:0.6461
epoch:40, loss:0.6461
epoch:50, loss:0.6461
epoch:60, loss:0.6461
epoch:70, loss:0.6461
epoch:80, loss:0.6461
epoch:90, loss:0.6461
epoch:100, loss:0.6461


In [15]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"Accuracy = {acc:.4f}")

Accuracy = 0.7444


In [16]:
torch.save(model, "logreg_model_heart.h5")

In [17]:
print(model.eval())
print(model.state_dict())

LogisticRegression(
  (linear): Linear(in_features=12, out_features=1, bias=True)
)
OrderedDict([('linear.weight', tensor([[ 0.1449, -0.1418, -0.1410,  0.0157,  0.0990, -0.2274, -0.1233, -0.1251,
         -0.2244,  0.0182, -0.2318,  0.1331]])), ('linear.bias', tensor([-0.2436]))])


In [18]:
#loading the model
loaded_model = torch.load('logreg_model_heart.h5')
print(loaded_model.eval())
print(loaded_model.state_dict())

LogisticRegression(
  (linear): Linear(in_features=12, out_features=1, bias=True)
)
OrderedDict([('linear.weight', tensor([[ 0.1449, -0.1418, -0.1410,  0.0157,  0.0990, -0.2274, -0.1233, -0.1251,
         -0.2244,  0.0182, -0.2318,  0.1331]])), ('linear.bias', tensor([-0.2436]))])


In [19]:
mean = X.mean()
std = X.std()
mod_X = (X - mean) / std

In [20]:
X = torch.from_numpy(X.astype(np.float32))
y = torch.from_numpy(y.astype(np.float32))

In [21]:
y = y.view(-1,1)

In [22]:
model2 = LogisticRegression(n_features)

In [23]:
num_epochs = 100
for epoch in range(num_epochs):
    y_predicted = model2(X)
    loss = criterion(y_predicted, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch + 1) % 10 == 0:
        print(f"epoch:{epoch + 1}, loss:{loss.item():.4f}")

epoch:10, loss:32.1070
epoch:20, loss:32.1070
epoch:30, loss:32.1070
epoch:40, loss:32.1070
epoch:50, loss:32.1070
epoch:60, loss:32.1070
epoch:70, loss:32.1070
epoch:80, loss:32.1070
epoch:90, loss:32.1070
epoch:100, loss:32.1070


In [24]:
with torch.no_grad():
    y_predicted = model2(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"Accuracy = {acc:.4f}")

Accuracy = 0.7333


In [25]:
torch.save(model, "logreg_model_heart_all.h5")

In [26]:
#loading the model
loaded_model = torch.load('logreg_model_heart_all.h5')